In [ ]:
!pip install huggingface_hub
!pip install datasets
!pip install ipywidgets
!pip install pandas
!git clone https://github.com/Chris1nexus/community-events.git
%cd community-events
!pip install .
%cd ..
!rm -rf community-events
!pip install wandb
#!huggingface-cli login
#!wandb login
#!accelerate config

In [13]:
from sklearn.model_selection import ParameterGrid
from subprocess import call
import os


'''
Reference parameters 

hyperparams_cyclegan = {
    'decay_epoch':[10] , #100
    'lr':[0.0002], 
    'n_residual_blocks':[9], 
    
    'lambda_cyc':[10.0], 
    'lambda_id':[5.0], 
    'beta1':[0.5], 
    'beta2':[0.999], 
    'batch_size':[8],
    
    'dataset_name':['Chris1/sim2real_gta5_to_cityscapes'],#['Chris1/NFT_cryptopunks_to_bored_apes'],  

    'channels':[3], 
    'checkpoint_interval':[10], 
    #'cpu':[False],     
    'epoch':[0], 
    #'fp16':[False], 
    'image_size':[256], 
    'mixed_precision':['no'], 
    'num_epochs':[20], 
    'num_workers':[8], 
    'organization_name':['Chris1'], 
    
    'wandb':[True],
    'push_to_hub':[True], 
    'model_name':['test'],
    
    'pytorch_dump_folder_path':[None], 
    'sample_interval':[100]
                     }
'''
hyperparams_cyclegan = { 
                        'lr':[0.0002], 
                        'num_epochs':[2], 
                        'decay_epoch':[1] , #100 
                        'n_residual_blocks':[9], 
                        'lambda_cyc':[10.0], 'lambda_id':[5.0], 
                        'beta1':[0.5], 'beta2':[0.999], 
                        'batch_size':[128],#[128], 
                        'dataset_name':['Chris1/NFT_cryptopunks_to_bored_apes_HQ'],#['Chris1/NFT_cryptopunks_to_bored_apes'], 
                        'channels':[3], 
                        'checkpoint_interval':[5], #'cpu':[False], 
                        'epoch':[0], #'fp16':[False], 
                        'image_size':[64], #256
                        'mixed_precision':['no'], 
                        'num_workers':[8], 
                        'organization_name':['Chris1'], 
                        'wandb':[True],
                        'push_to_hub':[True], 
                        'model_name':['test_cyclegan_punk_to_apes'],
                        'pytorch_dump_folder_path':['torch_dump_punk_to_apes'], 
                        'sample_interval':[30] }


In [15]:
import copy
import pandas as pd
EXPERIMENTS_DATA_CSV = 'experiments.csv'

if not os.path.exists(EXPERIMENTS_DATA_CSV):
    experiments_df = pd.DataFrame()
else:
    experiments_df = pd.read_csv(EXPERIMENTS_DATA_CSV)


EXPERIMENT_NAME = 'experiments_cyclegan_punk_to_apes'
    

for idx, grid in enumerate(ParameterGrid(hyperparams_cyclegan)):

        copy_grid = copy.deepcopy(grid)
        if 'dataset_name' in copy_grid:
            del copy_grid['dataset_name']
        if 'dataset' in copy_grid:
            del copy_grid['dataset']
        if 'output_dir' in copy_grid:
            del copy_grid['output_dir']
            
        experiment_name =  '--'.join([f'{k}__{v}'  for k,v in copy_grid.items() ])
        call_params = ['accelerate', 'launch',
                       '--config_file','~/.cache/huggingface/accelerate/default_config.yaml',
                       'train.py']
        
        
        experiment_id = len(experiments_df)
        grid['output_dir'] = experiment_name
        
        experiments_df = experiments_df.append(grid,ignore_index=True)
        grid['output_dir'] = f'experiments/{EXPERIMENT_NAME}/{experiment_id}'
        grid['pytorch_dump_folder_path'] = f'experiments/{grid["pytorch_dump_folder_path"]}_{experiment_id}'
        grid['model_name'] = f'{grid["model_name"]}_{experiment_id}'
        for k,v in grid.items():
            
             
            call_params.append(f'--{k}' )
            if k not in  ['cpu', 'wandb','fp16','push_to_hub']:
                call_params.append(f'{v}' )
        call(call_params)
        
        
        experiments_df.to_csv(EXPERIMENTS_DATA_CSV)

Namespace(batch_size=64, beta1=0.5, beta2=0.999, channels=3, checkpoint_interval=5, cpu=False, dataset_name='Chris1/NFT_cryptopunks_to_bored_apes_HQ', decay_epoch=1, epoch=0, fp16=False, image_size=64, lambda_cyc=10.0, lambda_id=5.0, lr=0.0002, mixed_precision='no', model_name='test_cyclegan_punk_to_apes_1', n_residual_blocks=9, num_epochs=2, num_workers=8, organization_name='Chris1', output_dir=PosixPath('experiments_cyclegan_punk_to_apes/1'), push_to_hub=True, pytorch_dump_folder_path=PosixPath('torch_dump_punk_to_apes_1'), sample_interval=100, wandb=True)
[Epoch 1/2] [Batch 140/141] [D loss: 0.180247] [G loss: 2.825500, adv: 0.492810, cycle: 0.161580, identity: 0.143379] ETA: 0:00:00.656010
